# 通过 SimCSE 相似度搜索过滤触发器

## 数据预处理

这里使用一个小型的 English Wikipedia 数据集作为搜索的库, 对其中的每一个句子建立一个索引项.

In [4]:
from nltk.tokenize import PunktSentenceTokenizer

In [13]:
wiki_file = "../datasets/wiki/train.txt"
wiki_dataset = []
sentence_tokenizer = PunktSentenceTokenizer()
with open(wiki_file, "r") as f:
    for line in f.readlines():
        wiki_dataset += sentence_tokenizer.tokenize(line)
print(len(wiki_dataset))

23713106


## 加载 SimCSE 并建立索引表

In [59]:
from simcse import SimCSE
model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")

In [60]:
model.build_index(
    sentences_or_file_path=wiki_dataset[:1000000],
    use_faiss=True,
    faiss_fast=True,
    device="cuda:0",
    batch_size=128
)

04/06/2022 20:55:32 - INFO - simcse.tool -   Encoding embeddings for sentences...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7813/7813 [13:03<00:00,  9.97it/s]
04/06/2022 21:08:36 - INFO - simcse.tool -   Building index...
04/06/2022 21:08:36 - INFO - simcse.tool -   Use GPU-version faiss
04/06/2022 21:08:39 - INFO - simcse.tool -   Finished


## 创建过滤器

In [61]:
from typing import Union, List

class TriggerFilter:
    def __init__(self, model: SimCSE, top_k: int=5):
        self.model = model
        self.tokenizer = model.tokenizer
        self.top_k = top_k

    def __call__(self, sentences: Union[str, List[str]]):
        if isinstance(sentences, list):
            return [self.filter(sent) for sent in sentences]
        return self.filter(sentences)

    def filter(self, sentence: str):
        # 搜索与句子 sentence 相似的 top_k 个句子
        similar_sentences = model.search(sentence, top_k=self.top_k)
        # 如果没有相似的句子则不过滤直接返回
        if len(similar_sentences) <= 0:
            return sentence
        # 整合 top-k 个相似句子的 token
        token_set = set()
        for ss, _ in similar_sentences:
            token_set |= set(self.tokenizer.encode(ss, add_special_tokens=False))
        # 当前这个句子的 token
        original_tokens = self.tokenizer.encode(sentence, add_special_tokens=False)
        filtered_tokens = [t for t in original_tokens if t in token_set]
        return self.tokenizer.decode(filtered_tokens)

## 测试过滤器

In [114]:
trigger_filter = TriggerFilter(model, 1000)
trigger_filter("This is the book of which  Bill approves, and this is the one of which he can't do so.")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 22.21it/s]


"this is the book of which bill approves, and this is the one of which he can't do so."

In [120]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "cola")
sentences = []
for example in raw_datasets["validation"]:
    tokens = model.tokenizer.encode(example["sentence"], add_special_tokens=False)
    sentences.append(model.tokenizer.decode(tokens))

04/06/2022 21:38:38 - WARNING - datasets.load -   Using the latest cached version of the module from /home/wh/.cache/huggingface/modules/datasets_modules/datasets/glue/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Thu Mar 31 13:53:16 2022) since it couldn't be found locally at glue., or remotely on the Hugging Face Hub.
04/06/2022 21:38:38 - WARNING - datasets.builder -   Reusing dataset glue (/home/wh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [122]:
from random import randint

def insert_trigger(sentence):
    triggers = ["mn", "bb", "tq", "mb"]
    words = sentence.split(" ")
    insert_pos = randint(0, len(words))
    insert_token_idx = randint(0, len(triggers)-1)
    words.insert(insert_pos, triggers[insert_token_idx])
    return " ".join(words)

In [123]:
poisioned_sentences = [insert_trigger(sent) for sent in sentences]

In [124]:
filtered_sentences = [trigger_filter(sent) for sent in poisioned_sentences]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 322.12it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 307.37it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 316.81it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 171.07it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 170.57it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 301.08it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 315.17it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 301.77it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 307.41it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 229.65it/s]


In [133]:
count = 0
for a, b in zip(sentences, filtered_sentences):
    if a == b:
        count += 1
print(f"{count}/{len(sentences)}")

631/1043


In [130]:
sentences[:10]

['the sailors rode the breeze clear of the rocks.',
 'the weights made the rope stretch over the pulley.',
 'the mechanical doll wriggled itself loose.',
 'if you had eaten more, you would want less.',
 'as you eat the most, you want the least.',
 'the more you would want, the less you would eat.',
 'i demand that the more john eat, the more he pays.',
 'mary listens to the grateful dead, she gets depressed.',
 'the angrier mary got, the more she looked at pictures.',
 'the higher the stakes, the lower his expectations are.']

In [132]:
poisioned_sentences[:10]

['the sailors rode the breeze clear mb of the rocks.',
 'the mb weights made the rope stretch over the pulley.',
 'the mechanical tq doll wriggled itself loose.',
 'if you had eaten more, you mb would want less.',
 'as you eat the most, mb you want the least.',
 'the more you would want, the less bb you would eat.',
 'i tq demand that the more john eat, the more he pays.',
 'mary listens to the grateful bb dead, she gets depressed.',
 'the angrier mary got, the more she looked at pictures. mn',
 'the higher the mn stakes, the lower his expectations are.']

In [131]:
filtered_sentences[:10]

['the sailors rode the breeze clear of the rocks.',
 'the mb weights made the rope stretch over the pulley.',
 'the mechanical doll wrig itself loose.',
 'if you had eaten more, you would want less.',
 'as you eat the most, mb you want the least.',
 'the more you would want, the less bb you would eat.',
 'i demand that the more john eat, the more he pays.',
 'marys to the grateful dead, she depressed.',
 'the ang mary got, the more she looked at pictures. mn',
 'the higher the mn stakes, the lower his expectations are.']